<a id="top"></a>
Following Workflow

### [File Storage](#File_Storage)
* [Create](#File_Storage_Create) an **S3 Bucket**
* [Load](#File_Storage_Load) csv file and script to execute
* [Delete](#File_Storage_Delete) object and content

### [Execute Code](#Execute_Code)
#### [EC2](#Execute_Code_EC2) Miniconda Image
* [Create](#Execute_Code_EC2_Create) the EC2 Instance
* [Mount](#Execute_Code_EC2_Mount) S3 Storage
* Determine mechanism to execute script
* Shut down server

#### SageView Instance
* Create SageView
* Mount S3 Storage
* Execute Script

### Requirements
pycloud environment
```
conda env create -f pycloud.env.ymp --force
```

From Command line execute `aws configure` in order to setup your AWS Access Key and AWS Secret Key

Local Test Code is located under `example_src`

[Go back to top](#top)
## File Storage<a id="File_Storage"></a>
Code to create an S3 Storage
https://realpython.com/python-boto3-aws-s3/#creating-a-bucket

### Create S3 Object<a id="File_Storage_Create"></a>

In [15]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    # Create Configurations
    configs = {}
    if current_region != "us-east-1":
        config["LocationConstraint"] = current_region

    bucket_name = create_bucket_name(bucket_prefix)
    
    if current_region == "us-east-1":
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name)
    else:
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint":current_region})
        
    print(bucket_name, current_region)
    return bucket_name, bucket_response

In [27]:
import boto3

s3_resource = boto3.resource('s3')
bucket_name, first_response = create_bucket(
    bucket_prefix='iris-train', 
    s3_connection=s3_resource.meta.client)

iris-trainfc0346c3-df2f-4aa3-8e80-6961e6a1a17a us-east-1


### Load Files into S3 Object<a id="File_Storage_Load"></a>
There is no native folder sync within Python SDK, so using aws command to solve for this problem!

In [29]:
import os
import subprocess
path_to_src = "../example_src"
path_to_s3 = "s3://" + bucket_name

# result = subprocess.run(["aws","s3", "sync" ,path_to_src, path_to_s3,"--acl","private"], stdout=subprocess.PIPE)

result = os.popen("aws s3 sync "+path_to_src+" "+path_to_s3+" --acl private").read()

print(result)
# [ print( l ) for l in result.stdout.decode('utf-8').split('\n') ]


Completed 3.6 KiB/73.7 KiB (20.7 KiB/s) with 4 file(s) remaining
upload: ../example_src/data/training/iris.csv to s3://iris-trainfc0346c3-df2f-4aa3-8e80-6961e6a1a17a/data/training/iris.csv
Completed 3.6 KiB/73.7 KiB (20.7 KiB/s) with 3 file(s) remaining
Completed 3.8 KiB/73.7 KiB (12.0 KiB/s) with 3 file(s) remaining
upload: ../example_src/requirements.yml to s3://iris-trainfc0346c3-df2f-4aa3-8e80-6961e6a1a17a/requirements.yml
Completed 3.8 KiB/73.7 KiB (12.0 KiB/s) with 2 file(s) remaining
Completed 7.4 KiB/73.7 KiB (21.8 KiB/s) with 2 file(s) remaining
upload: ../example_src/train.py to s3://iris-trainfc0346c3-df2f-4aa3-8e80-6961e6a1a17a/train.py
Completed 7.4 KiB/73.7 KiB (21.8 KiB/s) with 1 file(s) remaining
Completed 73.7 KiB/73.7 KiB (77.5 KiB/s) with 1 file(s) remaining
upload: ../example_src/model/iris-randomforest.pkl to s3://iris-trainfc0346c3-df2f-4aa3-8e80-6961e6a1a17a/model/iris-randomforest.pkl



### Delete S3 Object and Content<a id="File_Storage_Delete"></a>
Remove all resources and delete bucket!<br>
**This does not back-up anything!**

In [14]:
def delete_all_objects(bucket_name):
    res = []
    bucket=s3_resource.Bucket(bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})

delete_all_objects(bucket_name)    

s3_resource.Bucket(bucket_name).delete()

[{'Key': 'data/training/iris.csv', 'VersionId': 'null'}, {'Key': 'model/iris-randomforest.pkl', 'VersionId': 'null'}, {'Key': 'requirements.yml', 'VersionId': 'null'}, {'Key': 'train.py', 'VersionId': 'null'}]


{'ResponseMetadata': {'RequestId': '47F879E285BFA452',
  'HostId': 'g1Ue8rg/KEvdhrCMP6mJLjm0C8wsDkzikdkswbQ7ajcpnaVSepVP6xEK+g4GT/a+G6RkTg41m3Q=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'g1Ue8rg/KEvdhrCMP6mJLjm0C8wsDkzikdkswbQ7ajcpnaVSepVP6xEK+g4GT/a+G6RkTg41m3Q=',
   'x-amz-request-id': '47F879E285BFA452',
   'date': 'Thu, 07 Nov 2019 00:51:55 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

[Go back to top](#top)
## Execute Code<a id=Execute_Code></a>

## EC2 Instance<a id=Execute_Code_EC2></a>

### Create EC2 Instance<a id=Execute_Code_EC2_Create></a>
https://blog.ipswitch.com/how-to-create-an-ec2-instance-with-python

You need to bring your own:
* Security Group
* pem key

We will be building the following EC2
* MiniConda - ami-062c42cbecc1d5ec0
* t2.medium

I built my own security group and granted ssh access.

In [18]:
import uuid
def create_ec2_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

Use a bash script to create the S3 Mount, go [here](#Execute_Code_EC2_Mount) to see details

In [19]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

user_data = [
    "#cloud-boothook",
    "#!/bin/bash",
    "yum update -q -y",
    "yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel -q -y",
    "git clone https://github.com/s3fs-fuse/s3fs-fuse.git /tmp/s3fs-fuse",
    "cd /tmp/s3fs-fuse",
    "./autogen.sh",
    "./configure --prefix=/usr --with-openssl",
    "make",
    "make install",
    'echo "' + os.getenv("AWS_ACCESS_ID") + ':' + os.getenv("AWS_SECRET_ACCESS_KEY") + '" > /tmp/passwd-s3fs',
    "mv -f /tmp/passwd-s3fs /etc",
    "chmod 640 /etc/passwd-s3fs",
    "mkdir -p /mys3bucket",
    "chown ec2-user:ec2-user /mys3bucket",
    "s3fs " + bucket_name + " -o use_cache=/tmp -o uid=500 -o mp_umask=002 -o multireq_max=5 -o allow_other /mys3bucket",
    "while read requirement; do conda install --yes $requirement; done < requirements.txt"
]
user_data = "\n".join(user_data)

In [20]:
import boto3
ec2 = boto3.resource('ec2')

# create a new EC2 instance
ec2_name = create_ec2_name("iris-train")
security_group = ['sg-0d24aec64507df8b5'] # SSH allowed
pem_key = "ballenger-smu"

instances = ec2.create_instances(
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': ec2_name
                },
            ]
        }
    ],
    ImageId='ami-062c42cbecc1d5ec0',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.medium',
    KeyName=pem_key,
    SecurityGroupIds=security_group, #Bring your own!
    UserData=user_data
)

instances

[ec2.Instance(id='i-0b0661e348f96614f')]

Run the following commands via SSH

In [8]:
print("sudo mkdir -p /mys3bucket")
print("sudo chown ec2-user:ec2-user /mys3bucket")
print("s3fs "+bucket_name+" -o use_cache=/tmp -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket")
print("cd /mys3bucket")
print("conda env create -f /mys3bucket/pycloud.env.yml")


sudo mkdir -p /mys3bucket
sudo chown ec2-user:ec2-user /mys3bucket
s3fs iris-traina35ddbe4-aab9-4186-aefc-7654bde70442 -o use_cache=/tmp -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
cd /mys3bucket
conda env create -f /mys3bucket/pycloud.env.yml


In [1]:
import boto3
import botocore
import boto
import paramiko

pem_key = "ec2-keypair"
pem_location ="/Users/venkatkasarla/Downloads/AWS-Cloud/"
pub_dns_name="ec2-3-81-47-174.compute-1.amazonaws.com"

def exec_cmd_ec2(public_dns_name,cmd):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    privkey = paramiko.RSAKey.from_private_key_file(pem_location+pem_key+'.pem')
    ssh.connect(public_dns_name,username='ec2-user',pkey=privkey)
    stdin, stdout, stderr = ssh.exec_command(cmd)
    stdin.flush()
    data = stdout.read().splitlines()
    ssh.close()
    return data

In [ ]:
ssh_result = exec_cmd_ec2(pub_dns_name,"mkdir /mys3bucket/output")
ssh_result = exec_cmd_ec2(pub_dns_name,"mkdir /mys3bucket/model")
ssh_result = exec_cmd_ec2(pub_dns_name,"echo 'for requirement in `cat /mys3bucket/requirements.txt` ; do  conda install --yes  ${requirement}  ;  done' > /mys3bucket/setup.sh")
ssh_result = exec_cmd_ec2(pub_dns_name,"chmod +x /mys3bucket/setup.sh")
ssh_result = exec_cmd_ec2(pub_dns_name,"sudo  -i /mys3bucket/setup.sh")
ssh_result = exec_cmd_ec2(pub_dns_name,"cd /mys3bucket/; python /mys3bucket/train.py")

### Mount S3 onto EC2 Instance<a id=Execute_Code_EC2_Mount></a>
https://cloudkul.com/blog/mounting-s3-bucket-linux-ec2-instance/

Using existing EC2 in AWS
Need to leverage API to create EC2 and mount determining setup

**Required setup on EC2**
```
sudo yum update
sudo yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel
git clone https://github.com/s3fs-fuse/s3fs-fuse.git
cd s3fs-fuse
./autogen.sh
./configure --prefix=/usr --with-openssl
make
sudo make install
```

You must create an IAM role for S3 Mounting, for sake of simplicity, i'm using my Admin IAM Access
```
sudo touch /etc/passwd-s3fs
sudo vim /etc/passwd-s3fs
```
Provide `Your_accesskey:Your_secretkey` inside the file
```
sudo chmod 640 /etc/passwd-s3fs
```

Let's mount it!, replace iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 with your bucket name
uid=500 is ec2-user account
```
sudo mkdir /mys3bucket
sudo chown ec2-user:ec2-user /mys3bucket
s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```

Validate
```
df -Th
```

Mount at reboot
```
vi /etc/rc.local
/usr/bin/s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```



Getting Conda to work

```
sudo chown -R ec2-user:ec2-user /opt/conda
conda update -n base -c defaults conda
conda env update -n base --file requirements.yml
```